# Lab #3 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/basic-operations-workshop/blob/main/lab3.ipynb)
1. Install dependencies
2. Create a pinecone index 
3. Load public image dataset(fashion-mnist) and create vector embeddings from the dataset
4. Insert the fashion-mnist embeddings into Pinecone
5. Run a nearest neighbor search on a sample image that is not in the training dataset
6. Run a nearest neighbor search on 100 random test images that are not in the training dataset
7. Run a load test script to simulate 10 concurrent users querying the index
8. TEARDOWN: Delete the index 

# 1. Install Pinecone client 
Use the following shell command to install Pinecone:

In [1]:
!pip freeze > requirements.txt
!cat requirements.txt | xargs -n 1 pip uninstall -y
!pip install -U "pinecone-client" "python-dotenv" "torch" "torchvision" "pillow" "ftfy" "regex" "tqdm" "git+https://github.com/openai/clip.git" "datasets" "locust" "tensorflow==2.13"


Found existing installation: absl-py 1.4.0
Uninstalling absl-py-1.4.0:
  Successfully uninstalled absl-py-1.4.0
Found existing installation: aiohttp 3.8.5
Uninstalling aiohttp-3.8.5:
  Successfully uninstalled aiohttp-3.8.5
Found existing installation: aiosignal 1.3.1
Uninstalling aiosignal-1.3.1:
  Successfully uninstalled aiosignal-1.3.1
Found existing installation: appnope 0.1.3
Uninstalling appnope-0.1.3:
  Successfully uninstalled appnope-0.1.3
Found existing installation: asttokens 2.2.1
Uninstalling asttokens-2.2.1:
  Successfully uninstalled asttokens-2.2.1
Found existing installation: astunparse 1.6.3
Uninstalling astunparse-1.6.3:
  Successfully uninstalled astunparse-1.6.3
Found existing installation: async-timeout 4.0.2
Uninstalling async-timeout-4.0.2:
  Successfully uninstalled async-timeout-4.0.2
Found existing installation: attrs 23.1.0
Uninstalling attrs-23.1.0:
  Successfully uninstalled attrs-23.1.0
Found existing installation: backcall 0.2.0
Uninstalling backcall-0.

# 2. Create a pinecone index 
We will create an index that will be used to load/query a hugging face dataset.

In [3]:
from dotenv import load_dotenv
import os
import pinecone

load_dotenv('.env')

PINECONE_INDEX_NAME = os.environ['PINECONE_INDEX_NAME']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_ENVIRONMENT = os.environ['PINECONE_ENVIRONMENT']
METRIC = os.environ['METRIC']
DIMENSIONS = int(os.environ['DIMENSIONS'])

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone.create_index(PINECONE_INDEX_NAME, dimension=DIMENSIONS, metric=METRIC, pods=1, replicas=1, pod_type="s1")
pinecone.describe_index(PINECONE_INDEX_NAME)

ApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Thu, 03 Aug 2023 14:12:39 GMT', 'x-envoy-upstream-service-time': '411', 'content-length': '35', 'server': 'envoy'})
HTTP response body: index james-williams already exists


# 3. Load public image dataset(fashion-mnist) and create vector embeddings from the dataset

Use the following shell command to download the [fashion-mnist](https://huggingface.co/datasets/fashion_mnist) training dataset from Hugging Face so that we can create vector embeddings that uses a label(image class) as meta-data from this dataset. The meta-data labels mappings are:

| Label  | Description |
| ------ | ----------- |
| 0      | T-shirt/top |
| 1      | Trouser     |
| 2      | Pullover    |
| 3      | Dress       |
| 4      | Coat        |
| 5      | Sandal      |
| 6      | Shirt       |
| 7      | Sneaker     |
| 8      | Bag         |
| 9      | Ankle boot  |

The Fashion-MNIST dataset is a dataset of Zalando's article images, consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

The accuracy you can achieve depends on the model and the preprocessing steps you use. Here's a rough guideline for what you might expect with some classic machine learning algorithms:

1. **Random Forest:** Around 85-89% accuracy.
2. **Support Vector Machines (SVM):** Around 85-90% accuracy, depending on kernel and hyperparameters.
3. **k-Nearest Neighbors (k-NN):** Around 85-88% accuracy.
4. **Logistic Regression:** Around 82-85% accuracy.
5. **Gradient Boosting Machines (e.g., XGBoost):** Around 87-90% accuracy.

Keep in mind these numbers are approximate and can vary based on the exact preprocessing, feature extraction, and hyperparameter tuning you do. In general, deep learning models, especially Convolutional Neural Networks (CNNs), tend to perform better on image classification tasks like Fashion-MNIST, potentially reaching over 90-95% accuracy.

But for classic machine learning models, anything in the 85-90% range can be considered a reasonable result for the Fashion-MNIST dataset. It reflects a model that has learned something meaningful from the data but isn't necessarily state-of-the-art for this particular task.

In [1]:
from datasets import load_dataset
from tqdm.auto import tqdm  # progress bar
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import torch
import clip
import time
import numpy as np

#  Load the fashion-mnist dataset - only retrieve 6000 random images (10% of total dataset)
dataset = load_dataset("fashion_mnist")['train'].shuffle(seed=42).select(range(0,6000))
INDEX_NAMESPACE = "fashion-mnist-resnet"

label_descriptions = {0: "T-shirt/top", 
           1: "Trouser",
           2: "Pullover",
           3: "Dress",
           4: "Coat",
           5: "Sandal",
           6: "Shirt",
           7: "Sneaker",
           8: "Bag",
           9: "Ankle boot"}

print(f"using embedding model: {INDEX_NAMESPACE}")

model = ResNet50(weights='imagenet', include_top=False, pooling='avg') if INDEX_NAMESPACE == "fashion-mnist-resnet" else None

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device) if INDEX_NAMESPACE == "fashion-mnist-clip" else (model, None)
   
# Generate vector embeddings for each image in the dataset
id = 0
vectors = []
for img in tqdm(dataset, total=dataset.num_rows, desc='Images', position=0):
    with torch.no_grad():
        id += 1
        if INDEX_NAMESPACE == "fashion-mnist-clip":
            image_pp = preprocess(img['image']).unsqueeze(0).to(device)
            image_features = model.encode_image(image_pp)
            embedding = image_features.cpu().numpy().squeeze().tolist()
        elif INDEX_NAMESPACE == "fashion-mnist-resnet":
            image_array = np.array(img['image'], dtype='uint8')
            pil_image = Image.fromarray(image_array)
            pil_image_resized = pil_image.resize((224, 224))
            pil_image_rgb = pil_image_resized.convert('RGB')
            image_array = img_to_array(pil_image_rgb)
            image_array = np.expand_dims(image_array, axis=0)
            image_preprocessed = preprocess_input(image_array)
            embedding = model.predict(image_preprocessed, verbose=0).squeeze().tolist()

        meta_data = {"description": label_descriptions[img["label"]], "timestamp": time.time()}
        vectors.append({'id': str(id),
                        'values': embedding,
                        'metadata': meta_data})

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using embedding model: fashion-mnist-resnet


Images: 100%|██████████| 6000/6000 [06:24<00:00, 15.61it/s]


# 4. Insert the fashion-mnist embeddings into Pinecone

The best way to do bulk updates is by batching the dataset. We will also use a namespace for the data. 

In [4]:
from tqdm.auto import tqdm  # progress bar
import pinecone
import itertools

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

index = pinecone.Index(PINECONE_INDEX_NAME)

# Obtain the upsert embeddings in batches of 100
batch_size = 100
id = 0
for vector_batch in tqdm(chunks(vectors, batch_size=batch_size), total=(len(vectors) / batch_size)):
   index.upsert(vector_batch, namespace=INDEX_NAMESPACE)

100%|██████████| 60/60.0 [01:28<00:00,  1.47s/it]


# 5. Run a nearest neighbor search on a sample image that is not in the training dataset

Download a sneaker image file from github that we will use to run a query to see if pinecone search returns the correct description "Sneaker". 
You can change the top_k from 1 to 10 to 100 to see how the ANN results vary.

In [15]:
import pinecone
from PIL import Image
import torch
import clip
import requests
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image

print(f"using embedding model: {INDEX_NAMESPACE}")

model = ResNet50(weights='imagenet', include_top=False, pooling='avg') if INDEX_NAMESPACE == "fashion-mnist-resnet" else None

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device) if INDEX_NAMESPACE == "fashion-mnist-clip" else (model, None)

def image_to_embedding():
    
    url = "https://github.com/pinecone-io/basic-operations-workshop/blob/main/sneaker.png?raw=true"
    response = requests.get(url)
    with open("sneaker.png", "wb") as file:
      file.write(response.content)

    # Load and preprocess the image
    #image = preprocess(Image.open("./sneaker.png")).unsqueeze(0).to(device)

    if INDEX_NAMESPACE == "fashion-mnist-clip":
      image_pp = preprocess(Image.open("./sneaker.png")).unsqueeze(0).to(device)
      with torch.no_grad():
        embedding = model.encode_image(image_pp)
    elif INDEX_NAMESPACE == "fashion-mnist-resnet":
      img = image.load_img("./sneaker.png", target_size=(224, 224))
      x = image.img_to_array(img)
      x = np.expand_dims(x, axis=0)
      x = preprocess_input(x)
      embedding = model.predict(x).squeeze().tolist()
      print(embedding)
      #embedding = model.predict(image_preprocessed, verbose=0).squeeze().tolist()
    
    return embedding

index = pinecone.Index(PINECONE_INDEX_NAME)
top_k = 10

query_result = index.query(
  vector = image_to_embedding(),
  namespace=INDEX_NAMESPACE,
  top_k=top_k,
  include_values=False,
  include_metadata=True
)

top_k_contains = False
match_cnt = 0
miss_categories = set()

for match in query_result.matches:
  if match.metadata['description'] == "Sneaker":
    match_cnt += 1
    top_k_contains = True
  else:
    miss_categories.add(match.metadata['description'])

print(f"top_k contains matching result: {top_k_contains}")
print(f"top_k: {top_k} match percentage is: {match_cnt/top_k * 100}%")
print(f"Match miss categories: {miss_categories} exepected 'Sneaker'")

using embedding model: fashion-mnist-resnet
1/1 [==============================] - 0s 360ms/step
[0.20316220819950104, 1.2984994649887085, 0.32513999938964844, 0.11305668950080872, 0.0, 0.14795733988285065, 0.09701255708932877, 0.35932186245918274, 0.0, 0.0, 2.7978851795196533, 0.4478425681591034, 0.7082293033599854, 0.17005272209644318, 0.00974888727068901, 0.009415466338396072, 0.0, 0.029244206845760345, 0.03168197721242905, 0.0, 0.09717417508363724, 2.1337897777557373, 0.22135818004608154, 1.0173066854476929, 0.29625725746154785, 0.09623216837644577, 1.1826772689819336, 0.0, 0.0, 0.05531296506524086, 0.032861094921827316, 1.150726079940796, 0.43601733446121216, 0.19419020414352417, 1.676540493965149, 0.08897142112255096, 0.22746416926383972, 0.340557724237442, 0.06074795499444008, 0.008374467492103577, 0.8064901232719421, 1.1323214769363403, 0.03994488716125488, 0.01313010323792696, 1.9736357927322388, 0.08111770451068878, 0.0002879220701288432, 0.23172393441200256, 0.0, 0.267284899

# 6. Run a nearest neighbor search on 100 random test images that are not in the training dataset

Select 100 random test images. Keep in mind the model was NOT trained against these images. Obtain the percentage of pinecone queries that return the correct result in top_k. 

In [ ]:
import clip
import torch
import pinecone
from datasets import load_dataset
from tqdm.auto import tqdm 

test_dataset = load_dataset("fashion_mnist")['test'].shuffle(seed=42).select(range(0, 100))

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)

# Generate vector embeddings for each image in the dataset
test_vectors = []
for image in tqdm(test_dataset, total=test_dataset.num_rows):
    with torch.no_grad():
        image_pp = preprocess(image['image']).unsqueeze(0).to(device)
        image_features = model.encode_image(image_pp)
        embedding_numpy = image_features.cpu().numpy().squeeze().tolist()
        id += 1
        test_vectors.append({'embedding': embedding_numpy,
                        'description': label_descriptions[image["label"]]})

index = pinecone.Index(PINECONE_INDEX_NAME)
top_k = 10
top_k_contains_cnt = 0

for v in test_vectors:

  top_k_contains = False

  query_result = index.query(
    vector = v['embedding'],
    namespace=INDEX_NAMESPACE,
    top_k=top_k,
    include_values=False,
    include_metadata=True
  )
  
  for match in query_result.matches:
    if match.metadata['description'] == v['description']:
      top_k_contains = True

  if top_k_contains:
    top_k_contains_cnt += 1

print(f"top_k contains matching result: {top_k_contains_cnt / (len(test_vectors)) * 100}%")

# 7. Run a load test script to simulate 10 concurrent users querying the index

Locust.io is an open-source load testing tool written in Python. It allows you to define user behaviour with Python code and simulate millions of simultaneous users to bombard a system with traffic to test its resilience under heavy load. The (locustfile.py)[./locustfile.py] script re-uses the logic in step #6 to query pinecone. It has a custom event hook that denotes a failure if the top_k result set does not match the search image description. This script will likely fail with a low error rate but you can increase top_k to get a 100% pass rate. The locust summary includes P50 to P100 response time percentiles and QPS(req/s).

In [ ]:
%%bash
locust -f locustfile.py --headless -u 10 -r 1 --run-time 60s --host https://pinecone.io --only-summary

# 7. TEARDOWN: Delete the index 
# WARNING: This next step will delete the PINECONE_INDEX_NAME index and all data in it. DO NOT RUN THIS UNTIL YOU ARE READY OR MANUALLY REMOVE THE INDEX INSTEAD!!! 

In [ ]:
if PINECONE_INDEX_NAME in pinecone.list_indexes():
    pinecone.delete_index(PINECONE_INDEX_NAME)
    
pinecone.list_indexes()

In [5]:
!pip freeze > requirements.txt